In [ ]:
import numpy as np
import scipy.integrate as integrate
import attr
import matplotlib.pyplot as plt

# Helper class used as a reference

In [ ]:
@attr.s
class Rayleigh:
    delta = attr.ib(default=0.)
    ddelta = attr.ib(default=None)
    a = attr.ib(default=None)
    b = attr.ib(default=None)
    normalisation = attr.ib(default=None)
    
    def __attrs_post_init__(self):
        self.init()
    
    def init(self):
        self.ddelta = (1 - self.delta) / (1 + 0.5 * self.delta)
        self.a = 3. * self.ddelta / (16. * np.pi)
        self.b = (1. - self.ddelta) / (4. * np.pi)
        self.normalisation = self.integral()

    def eval(self, cos_theta):
        if self.delta == 0.:
            return self.a * (1. + np.square(cos_theta))
        else:
            return self.a * (1. + np.square(cos_theta)) + self.b
    
    def integral(self, bounds=[0, np.pi]):
        """Compute the integral of Planck's function on the spectral region of interest"""
        return integrate.quad(lambda x: self.eval(x), *bounds)[0]
    
    def pdf(self, cos_theta):
        return self.eval(cos_theta) / self.normalisation
    
    def cdf(self, cos_theta):
        """Return the cumulative distribution function for spectral values passed as the argument"""
        cdf = integrate.cumtrapz(y=self.pdf(cos_theta), x=cos_theta, initial=0.)
        return cdf / cdf[-1] # We don't forget to normalise to 1


In [ ]:
# Plot phase function pdf
x = np.linspace(-1, 1, 51)

plt.figure(figsize=(5,5))
for delta in np.linspace(0, 0.5, 11):
    plt.plot(x, Rayleigh(delta).pdf(x), label=f"{delta:0.2f}")
bottom, top = plt.ylim()
plt.ylim(-0.01, top)
plt.xlabel(r"cos $\theta$")
plt.title("Rayleigh phase function PDF")
plt.legend(loc="best", ncol=4, title="$\delta$")
plt.savefig("rayleigh_pdf.png")
plt.show()
plt.close()

In [ ]:
# Plot phase function CDF
x = np.linspace(-1, 1, 51)

plt.figure(figsize=(5, 5))
for delta in np.linspace(0, 0.5, 11):
    plt.plot(x, Rayleigh(delta).cdf(x), label=f"{delta:0.2f}")
plt.xlabel(r"cos $\theta$")
plt.title("Rayleigh phase function CDF")
plt.legend(loc="lower right", ncol=2, title="$\delta$")
plt.savefig("rayleigh_cdf.png")
plt.show()
plt.close()

# Mitsuba plugin check

In [ ]:
# Basic definitions
import mitsuba
import enoki as ek
mitsuba.set_variant("scalar_rgb")

from mitsuba.render import MediumInteraction3f, PhaseFunctionContext
from mitsuba.core import Frame3f

wi = [0, 0, 1]

def make_context(n):
    mi = MediumInteraction3f.zero(n)
    mi.wi = wi
    ek.set_slices(mi.wi, n)
    mi.sh_frame = Frame3f(-mi.wi)
    mi.wavelengths = []
    ctx = PhaseFunctionContext(None)
    return mi, ctx

def eval_pdf_mitsuba(phase, cos_thetas):
    mi, ctx = make_context(1)
    return np.array([phase.eval(ctx, mi, angles_to_vector(cos_theta, 0.))
                     for cos_theta in cos_thetas])

In [ ]:
import numpy as np
from mitsuba.core import Vector3f

def angles_to_vector(cos_theta, phi):
    sin_theta = np.sqrt(1.0 - cos_theta * cos_theta)
    sin_phi, cos_phi = np.sin(phi), np.cos(phi)
    return Vector3f(sin_theta * cos_phi, sin_theta * sin_phi, cos_theta)

for cos_theta, phi in [(1., 0.), (0., 0.), (-1., 0)]:
    print(f"angles_to_vector({cos_theta}, {phi}) = "
          f"{angles_to_vector(cos_theta, phi)}")

In [ ]:
from mitsuba.core.xml import load_string

delta = 0.25
r = Rayleigh(delta)
p = load_string(f"""
    <phase version="2.0.0" type="rayleigh">
        <float name="delta" value="{delta}"/>
    </phase>
""")

print(attr.asdict(r))
print(p)

cos_thetas = np.linspace(-1, 1, 51)
rayleigh_values_mitsuba = eval_pdf_mitsuba(p, cos_thetas)
rayleigh_values_reference = r.eval(cos_thetas)

for cos_theta, rayleigh_value_mitsuba, rayleigh_value_reference \
        in zip(cos_thetas, rayleigh_values_mitsuba, rayleigh_values_reference):
    print(f"{cos_theta:0.2f} | {rayleigh_value_mitsuba:0.4f} | {rayleigh_value_reference:0.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
plt.plot(cos_thetas, rayleigh_values_reference, label="Reference")
plt.plot(cos_thetas, rayleigh_values_mitsuba, linestyle="none", marker="o", label="Mitsuba")
plt.xlabel(r"cos $\theta$")
plt.title("Rayleigh phase function PDF ($\delta = 0$)")
plt.legend(loc="best", ncol=2)
plt.show()